In [3]:
from utils import *
import numpy as np
import torch.optim as optim
import ray
from ray import tune
import torch.nn as nn


In [2]:
X_train,X_val, X_test, y_train, y_val, y_test = load_data('FVC')
# print(X.shape, Y.shape)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

torch.Size([38, 155]) torch.Size([38, 1])
torch.Size([5, 155]) torch.Size([5, 1])
torch.Size([5, 155]) torch.Size([5, 1])


In [8]:
torch.manual_seed(42)
# Define model, loss function, and optimizer
model = MLP(input_size=X_train.shape[1], hidden_sizes=[50,10], output_size=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train and validate the model
num_epochs = 1
# train_losses = []
# val_losses = []
best_val_loss = float('inf')
ep =0
for epoch in range(num_epochs):
    
    # Training
    model.train()

    outputs = model(X_train)
    # outputs = outputs[:, 0].reshape(-1,1)
    print(outputs.shape, y_train.shape)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # train_losses.append(loss.item())
    
    # Validation
    model.eval()
    with torch.no_grad():
        outputs = model(X_val)
        val_loss = criterion(outputs, y_val)
        # val_losses.append(loss.item())

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            ep = epoch
            torch.save(model.state_dict(), 'best_model.pt') 

    if (epoch+1 ==1) or (epoch+1) % 500 == 0:
        print('Epoch [{}/{}], Train Loss: {:.4f}, Val Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item(), val_loss.item()))

# Evaluate 
model.load_state_dict(torch.load('best_model.pt'))
model.eval()
with torch.no_grad():
    print('val_loss', criterion(model(X_val), y_val).item())
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test)
    print('MSE Test Loss: {:.4f}'.format(test_loss.item()))
    print('MAPE Test Loss: {:.4f}'.format(mape_loss(y_pred, y_test).item())) 

torch.Size([38, 1]) torch.Size([38, 1])
torch.Size([38]) torch.Size([38])
torch.Size([38, 1]) torch.Size([38, 1])
Epoch [1/1], Train Loss: 9.7188, Val Loss: 10.3862
val_loss 10.386189460754395
MSE Test Loss: 8.4290
MAPE Test Loss: 94.6442


In [38]:
ep

196

In [39]:
best_val_loss

0.04501364752650261

In [68]:
import ray
from ray import tune

def train_mlp(config, checkpoint_dir=None):
    X_train,X_val, X_test, y_train, y_val, y_test = load_data('FVC')
    best_val_loss = float('inf')

    model = MLP(input_size=X_train.shape[1], hidden_sizes=config["hidden_size"], output_size=1)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    num_epochs = 500
    for epoch in range(num_epochs):
      # Training
      model.train()
      outputs = model(X_train)
      loss = criterion(outputs, y_train)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      tune.report(loss=loss.item())
      # Validation
      model.eval()
      with torch.no_grad():
            outputs = model(X_val)
            val_loss = criterion(outputs, y_val)
            if val_loss.item() < best_val_loss:
                best_val_loss = val_loss

      # tune.report(val_loss = best_val_loss)
      tune.report(val_loss = val_loss.item())
      # tune.report(best_val_loss = best_val_loss.item())


In [75]:
config = {
    "hidden_size": tune.choice([[50],[50,10], [75,35,15,5],[60,30,5] ]),
    # "lr": tune.loguniform(1e-5, 1e-1),
    'lr':tune.choice([0.01, 0.005, 0.001]),
    # "num_epochs": tune.choice([2000])
}


ray.shutdown()
# Initialize Ray
ray.init()


analysis = tune.run(
    train_mlp,
    config=config,
    num_samples=10,
    progress_reporter=tune.CLIReporter()
)

2023-04-26 14:56:51,232	INFO worker.py:1625 -- Started a local Ray instance.
2023-04-26 14:56:52,119	WARNING callback.py:142 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


== Status ==
Current time: 2023-04-26 14:56:54 (running for 00:00:02.79)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/16 CPUs, 0/2 GPUs
Result logdir: /home/akbar/ray_results/train_mlp_2023-04-26_14-56-52
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-----------------------+----------+--------------------+-----------------+-------+
| Trial name            | status   | loc                | hidden_size     |    lr |
|-----------------------+----------+--------------------+-----------------+-------|
| train_mlp_7a547_00000 | RUNNING  | 10.7.44.21:1033366 | [75, 35, 15, 5] | 0.001 |
| train_mlp_7a547_00001 | PENDING  |                    | [75, 35, 15, 5] | 0.01  |
| train_mlp_7a547_00002 | PENDING  |                    | [75, 35, 15, 5] | 0.01  |
| train_mlp_7a547_00003 | PENDING  |                    | [50, 10]        | 0.01  |
| train_mlp_7a547_00004 | PENDING  |                    | [50]            | 0.005 |
| train_mlp_7a547_00005 | PENDING  |                    | [7

Trial name,date,done,hostname,iterations_since_restore,loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_mlp_7a547_00000,2023-04-26_14-56-54,False,cvig,1,10.2943,10.7.44.21,1033366,0.0610158,0.0610158,0.0610158,1682501214,1,7a547_00000


2023-04-26 14:57:06,775	INFO tune.py:945 -- Total run time: 14.67 seconds (14.63 seconds for the tuning loop).


== Status ==
Current time: 2023-04-26 14:57:06 (running for 00:00:14.64)
Using FIFO scheduling algorithm.
Logical resource usage: 0/16 CPUs, 0/2 GPUs
Result logdir: /home/akbar/ray_results/train_mlp_2023-04-26_14-56-52
Number of trials: 10/10 (10 TERMINATED)
+-----------------------+------------+--------------------+-----------------+-------+--------+------------------+------------+
| Trial name            | status     | loc                | hidden_size     |    lr |   iter |   total time (s) |   val_loss |
|-----------------------+------------+--------------------+-----------------+-------+--------+------------------+------------|
| train_mlp_7a547_00000 | TERMINATED | 10.7.44.21:1033366 | [75, 35, 15, 5] | 0.001 |   1000 |          6.8099  |  0.100491  |
| train_mlp_7a547_00001 | TERMINATED | 10.7.44.21:1033427 | [75, 35, 15, 5] | 0.01  |   1000 |          7.99999 |  0.0604842 |
| train_mlp_7a547_00002 | TERMINATED | 10.7.44.21:1033428 | [75, 35, 15, 5] | 0.01  |   1000 |          8.

In [77]:
# Print the best hyperparameters found by Ray Tune
best_config = analysis.get_best_config(metric="val_loss", mode="min")
print("Best config:", best_config)


Best config: {'hidden_size': [50], 'lr': 0.005}


In [3]:
best_config= {'hidden_size': [50], 'lr': 0.005}

In [4]:
import tempfile
import torch.utils.tensorboard as tb

In [8]:
log_dir  = 'D:\iitgn\Thesis\Spiro_Mask2'
LOGGER = tb.SummaryWriter(log_dir + '/train', flush_secs = 1)

In [9]:
from torch.utils.tensorboard import SummaryWriter
torch.manual_seed(42)
# Define the model
model = MLP(input_size=X_train.shape[1], hidden_sizes=best_config["hidden_size"], output_size=1)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=best_config["lr"])

# Train and validate the model
# num_epochs = 5000
# train_losses = []
# val_losses = []
best_val_loss = float('inf')
num_epochs = 200 
for epoch in range(num_epochs):
    
    # Training
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    LOGGER.add_scalar('Train Loss', loss, epoch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # train_losses.append(loss.item())
    
    # Validation
    model.eval()
    with torch.no_grad():
        outputs = model(X_val)
        val_loss = criterion(outputs, y_val)
        # val_losses.append(loss.item())

        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            torch.save(model.state_dict(), 'best_model_hyper.pt') 
    
    # Print progress
    if (epoch+1 ==1) or (epoch+1) % 200 == 0:
        print('Epoch [{}/{}], Train Loss: {:.4f}, Val Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item(), val_loss.item()))

# Evaluate the best model on the test set
model.load_state_dict(torch.load('best_model_hyper.pt'))
model.eval()
with torch.no_grad():
    print('val_loss', criterion(model(X_val), y_val).item())
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test)
    print('MSE Test Loss: {:.4f}'.format(test_loss.item()))
    print('MAPE Test Loss: {:.4f}'.format(mape_loss(y_pred, y_test).item())) 

# Close the SummaryWriter instance
LOGGER.close() 

Epoch [1/200], Train Loss: 10.3959, Val Loss: 11.1884
Epoch [200/200], Train Loss: 0.0545, Val Loss: 0.0951
val_loss 0.09511382132768631
MSE Test Loss: 0.0287
MAPE Test Loss: 5.1128


In [1]:
%load_ext tensorboard
%tensorboard --logdir=D:\iitgn\Thesis\Spiro_Mask2\train

Reusing TensorBoard on port 6006 (pid 2132), started 8:07:10 ago. (Use '!kill 2132' to kill it.)